# Experiments with Non-separable Functions

## Performance on Non-separable Functions

We evaluate DLT on challenging non-separable convex functions where classical methods struggle. Unlike separable functions where $f(x) = \sum_i f_i(x_i)$, these functions have coupled variables requiring the network to learn complex interactions.

## Test Functions and Domains

### 1. Quadratic with Random SPD Matrix
- **Function**: $f(x) = \frac{1}{2}x^\top Q x$ where $Q$ is random SPD with $\kappa \approx 10^2$
- **Domain $C$**: $\mathcal{N}(0,1)^d$ (sampled as $\approx[-3,3]^d$)
- **Dual Domain $D$**: $\approx[-3,3]^d$

### 2. Exponential-minus-Linear  
- **Function**: $f(x) = e^{\langle a,x\rangle} - \langle b,x\rangle$ where $\|a\|=\|b\|=1$
- **Domain $C$**: $\mathcal{N}(0,1)^d$ (sampled as $\approx[-3,3]^d$)
- **Dual Domain $D$**: $\approx[e^{-3\sqrt{d}}, e^{3\sqrt{d}}]^d$

### 3. Pre-trained ICNN
- **Function**: $f(x) = \text{ICNN}_{2\text{-layer}}(x)$ with 128 hidden units
- **Domain $C$**: $\approx[-3s,3s]^d$ where $s \in [0.1,10]$ (auto-scaled)
- **Dual Domain $D$**: $\approx[-3,3]^d$

### 4. Coupled Soft-plus
- **Function**: $f(x) = \sum_{i<j}\log(1+e^{x_i+x_j})$ with $O(d^2)$ interactions
- **Domain $C$**: $[-1.5,1.5]^d$ (uniform)
- **Dual Domain $D$**: $\approx[0, d-1]^d$

### Function Details:

1. **Quadratic SPD**: Non-separable through random positive definite matrix $Q$ with controlled condition number
2. **Exp-Minus-Linear**: Sampled unit vectors $a, b \sim \mathcal{N}(0, I_d)$ normalized to unit sphere
3. **Pre-trained ICNN**: 2-layer network (128 units/layer) frozen after training on quadratic target
4. **Coupled Soft-plus**: Explicitly couples all variable pairs with $\binom{d}{2}$ interaction terms

## Experimental Results

**Table: DLT Performance on Non-separable Functions (ResNet, 5 trials)**

| Function | $d=20$ RMSE | $d=20$ Time (s) | $d=50$ RMSE | $d=50$ Time (s) |
|----------|-------------|-----------------|-------------|-----------------|
| Quadratic SPD | 7.37e-3 ± 7.0e-3 | 516 ± 1 | 4.04e-3 ± 2.5e-3 | 1336 ± 0.1 |
| Exp-minus-Linear | 1.01e-1 ± 9.6e-2 | 530 ± 2 | 3.01e-2 ± 1.6e-2 | 1333 ± 0.4 |
| 2-layer ICNN | 1.30e-3 ± 9.0e-4 | 609 ± 2 | 2.69e-4 ± 1.0e-5 | 1361 ± 0.5 |
| Coupled Soft-plus | 8.19e-3 ± 7.3e-3 | 1126 ± 5 | 2.37e-1 ± 2.2e-2 | 2857 ± 3 |

**Architecture**: ResNet with two blocks of size 128

**Key Finding**: DLT maintains consistent accuracy across all non-separable function types, successfully learning complex variable interactions that classical methods cannot handle.

In [ ]:
# dlt_resnet_suite_v9_smoke.py — ResNet-only, implicit (DLT-style) learning of f* for:
#   • Quadratic (random SPD)
#   • Exp−Linear (unit a & unit b)
#   • 2‑layer ICNN (pre‑learned to quadratic, then frozen)
#   • Coupled Soft‑plus (chunked O(d^2))
#
# This file includes a quick smoke test at the bottom for dims {5,10}. Comment it out for full runs.

import os, sys, time, math, argparse, copy
from dataclasses import dataclass
from typing import Optional, List, Dict, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ========================= Device & seed =========================
def get_device():
    if torch.cuda.is_available(): return torch.device("cuda")
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available(): return torch.device("mps")
    return torch.device("cpu")

def set_seed(seed:int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# ========================= SPD helpers ===========================
@torch.no_grad()
def make_spd(d:int, seed:int, norm:str="spectral", ridge:float=1e-2, device=None):
    torch.manual_seed(seed)
    A = torch.randn(d, d, dtype=torch.float32, device=device)
    Q = A.t() @ A
    if norm == "spectral":
        lam = torch.linalg.eigvalsh(Q)
        lam_max = lam.max().clamp_min(1e-12)
        Q = Q / lam_max
    elif norm == "mean":
        Q = Q / (torch.trace(Q) / float(d)).clamp_min(1e-12)
    else:
        raise ValueError(f"unknown spd_norm '{norm}'")
    Q = Q + ridge * torch.eye(d, device=device)
    return Q

@torch.no_grad()
def spd_stats(Q:torch.Tensor)->Dict[str,float]:
    lam = torch.linalg.eigvalsh(Q)
    return {"lambda_min": float(lam.min()),
            "lambda_max": float(lam.max()),
            "kappa": float((lam.max()/lam.min()).item()),
            "trace": float(torch.trace(Q))}

# ========================= Tasks ================================
@dataclass
class QuadSPD:
    d:int; Q:torch.Tensor; U:torch.Tensor; lam_inv_sqrt:torch.Tensor
    def sample_x(self,B:int): return torch.randn(B, self.d, device=self.Q.device)
    def f(self,x): return 0.5*torch.sum((x @ self.Q) * x, dim=-1)
    def grad(self,x): return x @ self.Q
    def z_from_y(self,y): return (y @ self.U) * self.lam_inv_sqrt

@dataclass
class ExpMinusLin:
    d:int; a:torch.Tensor; b:torch.Tensor
    def sample_x(self,B:int): return torch.randn(B, self.d, device=self.a.device)
    def f(self,x):
        ax = x @ self.a
        return torch.exp(ax) - (x @ self.b)
    def grad(self,x):
        ax = x @ self.a
        return torch.exp(ax)[:,None]*self.a[None,:] - self.b[None,:]
    def z_from_y(self,y):
        alpha = ((y + self.b[None,:]) @ self.a)[:, None].clamp_min(1e-12)
        return torch.cat([alpha, torch.log(alpha)], dim=-1)

# ---- Trainable 2-layer ICNN (pre-learning target) ----
class ICNN2Trainable(nn.Module):
    """
    2-layer ICNN with softplus; convexity via softplus(raw) on z->z and z->out weights.
    """
    def __init__(self, d:int, h1:int=128, h2:int=128, seed:int=0, device=None):
        super().__init__()
        g = torch.Generator(device=device); g.manual_seed(seed)
        self.U1 = nn.Parameter(torch.randn(d, h1, generator=g, device=device)/math.sqrt(d))
        self.U2 = nn.Parameter(torch.randn(d, h2, generator=g, device=device)/math.sqrt(d))
        self.b1 = nn.Parameter(torch.zeros(h1, device=device))
        self.b2 = nn.Parameter(torch.zeros(h2, device=device))
        self.raw_W21 = nn.Parameter(torch.randn(h1, h2, generator=g, device=device)*0.5)
        self.raw_wout= nn.Parameter(torch.randn(h2, 1, generator=g, device=device)*0.5)
        self.uout = nn.Parameter(torch.randn(d, 1, generator=g, device=device)/math.sqrt(d))
    def forward(self, x):
        W21 = F.softplus(self.raw_W21)    # nonnegative
        wout= F.softplus(self.raw_wout)   # nonnegative
        z1 = F.softplus(x @ self.U1 + self.b1)
        z2 = F.softplus(z1 @ W21 + x @ self.U2 + self.b2)
        return (z2 @ wout).squeeze(-1) + (x @ self.uout).squeeze(-1)

@dataclass
class ICNNTask:
    d:int; net:nn.Module; device:torch.device
    x_scale:float; y_mu:torch.Tensor; Wy_U:torch.Tensor; Wy_diag_inv_sqrt:torch.Tensor
    def sample_x(self,B:int): return self.x_scale * torch.randn(B, self.d, device=self.device)
    def f(self,x): return self.net(x)
    def grad(self,x):
        # Enable grad on x ONLY to get y, then detach. Never backprop through f.
        if not x.requires_grad:
            x = x.detach().requires_grad_(True)
        fx = self.net(x)
        (gy,) = torch.autograd.grad(fx.sum(), x, create_graph=False, retain_graph=False)
        return gy.detach()
    def z_from_y(self,y): return ((y - self.y_mu[None,:]) @ self.Wy_U) * self.Wy_diag_inv_sqrt

# ---- Coupled Soft-plus with chunked O(d^2) kernel ----
def softplus_pairs_f_grad(X:torch.Tensor, block:int=64):
    """
    f(x)=sum_{i<j} softplus(x_i+x_j)
    Compute f and ∇f in O(d^2) time with O(B*block) memory.
    """
    B, d = X.shape
    f = X.new_zeros(B)
    g = X.new_zeros(B, d)
    for i in range(d-1):
        xi = X[:, i:i+1]
        j = i + 1
        while j < d:
            J = slice(j, min(d, j+block))
            z = xi + X[:, J]
            sp = F.softplus(z)
            s  = torch.sigmoid(z)
            f += sp.sum(dim=1)
            g[:, i] += s.sum(dim=1)
            g[:, J] += s
            j += block
    return f, g

@dataclass
class SoftplusPairsTask:
    d:int; device:torch.device; x_scale:float; block:int
    y_mu:torch.Tensor; Wy_U:torch.Tensor; Wy_diag_inv_sqrt:torch.Tensor
    def sample_x(self,B:int): return self.x_scale * torch.randn(B, self.d, device=self.device)
    def f(self,x): f,_ = softplus_pairs_f_grad(x, block=self.block); return f
    def grad(self,x): _,g = softplus_pairs_f_grad(x, block=self.block); return g
    def z_from_y(self,y): return ((y - self.y_mu[None,:]) @ self.Wy_U) * self.Wy_diag_inv_sqrt

# ========================= Probing & whitening ===================
def icnn_probe_y_stats(net:nn.Module, d:int, device, x_scale:float, nsamp:int=16384):
    """Uses autograd to compute y; then detaches for stats."""
    B = 4096
    iters = max(1, (nsamp + B - 1)//B)
    norms = []
    y_sum = torch.zeros(d, device=device)
    yy_sum = torch.zeros(d, d, device=device)
    n = 0
    for _ in range(iters):
        b = min(B, nsamp - n)
        x = (x_scale * torch.randn(b, d, device=device)).requires_grad_(True)
        fx = net(x)
        (y,) = torch.autograd.grad(fx.sum(), x, create_graph=False, retain_graph=False)
        y = y.detach()
        norms.append(torch.linalg.norm(y, dim=1))
        y_sum += y.sum(dim=0)
        yy_sum += y.t() @ y
        n += b
    norms = torch.cat(norms, dim=0)
    mu = y_sum / n
    cov = (yy_sum / n) - mu[:,None] @ mu[None,:]
    eig = torch.linalg.eigvalsh(cov)
    return {"med_norm": float(torch.median(norms)),
            "mu": mu, "cov": cov,
            "eig_min": float(eig.min()), "eig_max": float(eig.max())}

@torch.no_grad()
def softplus_probe_y_stats(d:int, device, x_scale:float, block:int, nsamp:int=16384):
    B = 2048
    iters = max(1, (nsamp + B - 1)//B)
    y_sum = torch.zeros(d, device=device)
    yy_sum = torch.zeros(d, d, device=device)
    n = 0
    for _ in range(iters):
        b = min(B, nsamp - n)
        x = x_scale * torch.randn(b, d, device=device)
        _, y = softplus_pairs_f_grad(x, block=block)
        y_sum += y.sum(dim=0)
        yy_sum += y.t() @ y
        n += b
    mu = y_sum / n
    cov = (yy_sum / n) - mu[:,None] @ mu[None,:]
    eig = torch.linalg.eigvalsh(cov)
    return {"mu": mu, "cov": cov, "eig_min": float(eig.min()), "eig_max": float(eig.max())}

# ========================= Target scaler =========================
class TargetScaler:
    def __init__(self, momentum:float=0.98, eps:float=1e-8):
        self.momentum, self.eps = momentum, eps
        self.mu = None; self.sig = None
    @torch.no_grad()
    def fit_batch(self, t:torch.Tensor):
        bmu = t.mean()
        bsd = t.std(unbiased=False).clamp_min(self.eps)
        if self.mu is None: self.mu, self.sig = bmu, bsd
        else:
            self.mu  = self.momentum*self.mu  + (1-self.momentum)*bmu
            self.sig = self.momentum*self.sig + (1-self.momentum)*bsd
        return (t - self.mu)/self.sig, float(self.mu), float(self.sig)
    @torch.no_grad()
    def denorm(self, tzn:torch.Tensor):
        if self.mu is None or self.sig is None: return tzn
        return tzn*self.sig + self.mu

# ========================= ResNet learner ========================
class PreActBlock(nn.Module):
    def __init__(self, dim:int, hidden:int, alpha:float=0.5):
        super().__init__()
        self.ln1 = nn.LayerNorm(dim)
        self.fc1 = nn.Linear(dim, hidden)
        self.act = nn.GELU()
        self.ln2 = nn.LayerNorm(hidden)
        self.fc2 = nn.Linear(hidden, dim)
        self.alpha = alpha
        nn.init.xavier_uniform_(self.fc1.weight); nn.init.zeros_(self.fc1.bias)
        nn.init.xavier_uniform_(self.fc2.weight, gain=0.1); nn.init.zeros_(self.fc2.bias)
    def forward(self, x):
        y = self.fc1(self.ln1(x)); y = self.act(y); y = self.fc2(self.ln2(y))
        return x + self.alpha*y

class ResNetDLT(nn.Module):
    def __init__(self, in_dim:int, width:int, depth:int, square_feat:bool=True, use_layernorm:bool=False):
        super().__init__()
        self.square = square_feat
        eff_in = in_dim*(2 if self.square else 1)
        self.in_ln = nn.LayerNorm(eff_in) if use_layernorm else None
        self.embed = nn.Linear(eff_in, width)
        self.blocks = nn.ModuleList([PreActBlock(width, width) for _ in range(depth)])
        self.head_ln = nn.LayerNorm(width)
        self.head = nn.Linear(width, 1)
        nn.init.xavier_uniform_(self.embed.weight); nn.init.zeros_(self.embed.bias)
        nn.init.xavier_uniform_(self.head.weight);  nn.init.zeros_(self.head.bias)
    def forward(self, z):
        if self.square: z = torch.cat([z, z*z], dim=-1)
        if self.in_ln is not None: z = self.in_ln(z)
        h = self.embed(z)
        for b in self.blocks: h = b(h)
        return self.head(self.head_ln(h)).squeeze(-1)

def resnet_size_for_dim(dim:int)->Tuple[int,int]:
    if dim <= 10:  return 384, 4
    if dim <= 20:  return 512, 5
    if dim <= 50:  return 768, 6
    if dim <= 100: return 1024, 7
    return 1280, 8

# ========================= Batch rules & loss ====================
def macro_batch_size(d:int)->int:
    return max(600, 64*d)  # single fresh macro-batch per step

def huber_loss(pred, target, delta:float=1.0):
    return F.huber_loss(pred, target, delta=delta, reduction="mean")

# ========================= ICNN pre-learning =====================
def prelearn_icnn_to_quadratic(net:ICNN2Trainable,
                               d:int, device,
                               steps:int=500, lr:float=1e-3, batch:int=2048,
                               spd_norm:str="spectral", ridge:float=1e-2,
                               seed:int=12345):
    Q = make_spd(d, seed=seed, norm=spd_norm, ridge=ridge, device=device)
    stats = spd_stats(Q)
    opt = optim.AdamW(net.parameters(), lr=lr, weight_decay=1e-4)
    sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=steps, eta_min=1e-5)
    print(f"ICNN pre-learn → quadratic: steps={steps}, batch={batch}, lr={lr:.2e}, "
          f"||Q||₂≈{stats['lambda_max']:.3f}, κ={stats['kappa']:.2f}, ridge={ridge:g}")
    t0 = time.time()
    for s in range(1, steps+1):
        x = torch.randn(batch, d, device=device)
        target = 0.5*torch.sum((x @ Q)*x, dim=-1)
        pred = net(x)
        loss = F.mse_loss(pred, target)
        opt.zero_grad(set_to_none=True); loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        opt.step(); sched.step()
        if s % max(1, steps//5) == 0 or s == steps:
            with torch.no_grad():
                xe = torch.randn(batch, d, device=device)
                te = 0.5*torch.sum((xe @ Q)*xe, dim=-1)
                pe = net(xe)
                rmse = torch.sqrt(F.mse_loss(pe, te)).item()
            print(f"[pre {100*s/steps:5.1f}%] train MSE={loss.item():.3e} | eval RMSE={rmse:.3e}")
    print(f"ICNN pre-learn done in {time.time()-t0:.1f}s")

# ========================= Macro/mini sample =====================
def compute_macro_batch(obj, B:int):
    """
    Produce a single macro-/mini-batch (fresh randomness) and targets with *no gradient through f*.
    Returns (y_detached, t_detached, x_detached).
    """
    with torch.enable_grad():
        x = obj["sample_x"](B)
        x.requires_grad_(True)
        y = obj["grad"](x)          # ICNN task returns y.detach(); others are analytic
        y = y.detach()
        x_det = x.detach()
    with torch.no_grad():
        t = (x_det*y).sum(dim=-1) - obj["f"](x_det)
    return y, t, x_det

# ========================= Evaluation ============================
def evaluate_generic(model:nn.Module,
                     task_name:str,
                     obj,
                     z_map,
                     tscaler:Optional[TargetScaler],
                     device,
                     zdim:int,
                     test_B:int=8192) -> Dict[str, float]:
    y, t, x = compute_macro_batch(obj, test_B)  # detached
    with torch.no_grad():
        z = z_map(y, x)
        pred_zn = model(z)
        pred = tscaler.denorm(pred_zn) if tscaler is not None else pred_zn
        mse  = torch.mean((pred - t)**2).item()
        rmse = math.sqrt(mse)
        mae  = torch.mean((pred - t).abs()).item()
        t_std = t.std(unbiased=False).item()
        rel = rmse / max(t_std, 1e-8)
    return {"rmse":rmse, "mae":mae, "rel_rmse":rel, "t_std":t_std}

# ========================= Task builders =========================
def build_task_quad_spd(d:int, args, device):
    Q = make_spd(d, seed=args.seed + d*17, norm=args.spd_norm, ridge=args.ridge, device=device)
    stats = spd_stats(Q)
    lam, U = torch.linalg.eigh(Q)
    lam_inv_sqrt = (1.0 / torch.sqrt(lam.clamp_min(1e-12))).to(device)
    quad = QuadSPD(d=d, Q=Q, U=U, lam_inv_sqrt=lam_inv_sqrt)
    obj = {"sample_x": quad.sample_x, "f": quad.f, "grad": quad.grad,
           "z_from_y": lambda y,x: quad.z_from_y(y), "z_dim": d}
    meta = (f"Q stats: ||Q||₂≈{stats['lambda_max']:.3f}, κ={stats['kappa']:.2f}, trace={stats['trace']:.1f} | "
            f"features: z=Q^(-1/2) y (+ z^2 in model)")
    return obj, meta

def build_task_exp_minus_lin(d:int, args, device):
    g = torch.Generator(device=device); g.manual_seed(args.seed + d*31)
    a_raw = torch.randn(d, generator=g, device=device)
    b_raw = torch.randn(d, generator=g, device=device)
    a = a_raw / (a_raw.norm() + 1e-12)  # unit vector (normalized)
    b = b_raw / (b_raw.norm() + 1e-12)  # unit vector (normalized)
    task = ExpMinusLin(d=d, a=a, b=b)
    obj = {"sample_x": task.sample_x, "f": task.f, "grad": task.grad,
           "z_from_y": lambda y,x: task.z_from_y(y), "z_dim": 2}
    meta = (f"Exp−Linear: ||a||=1, ||b||=1 (both normalized) | features: z=[⟨y+b,a⟩, log⟨y+b,a⟩] (+ squares)")
    return obj, meta

def build_task_rand_icnn2(d:int, args, device):
    net = ICNN2Trainable(d=d, h1=128, h2=128, seed=args.seed + d*53, device=device)
    if args.icnn_prelearn_steps > 0:
        prelearn_icnn_to_quadratic(net, d, device,
                                   steps=args.icnn_prelearn_steps,
                                   lr=args.icnn_prelearn_lr,
                                   batch=args.icnn_prelearn_batch,
                                   spd_norm=args.icnn_prelearn_spd_norm,
                                   ridge=args.icnn_prelearn_ridge,
                                   seed=args.seed + 777*d)
    for p in net.parameters(): p.requires_grad_(False)
    net.eval()
    # Auto-scale then dual whitening
    probe1 = icnn_probe_y_stats(net, d, device, x_scale=1.0, nsamp=min(args.icnn_whiten_samples, 32768))
    med = probe1["med_norm"]; target = args.icnn_target_y_med
    scale = float(np.clip(target / max(med, 1e-6), 0.1, 10.0))
    probe2 = icnn_probe_y_stats(net, d, device, x_scale=scale, nsamp=args.icnn_whiten_samples)
    mu_y = probe2["mu"]; cov_y = probe2["cov"] + args.icnn_cov_reg * torch.eye(d, device=device)
    lam, U = torch.linalg.eigh(cov_y); lam_inv_sqrt = (1.0 / torch.sqrt(lam.clamp_min(1e-12))).to(device)
    task = ICNNTask(d=d, net=net, device=device, x_scale=scale, y_mu=mu_y,
                    Wy_U=U, Wy_diag_inv_sqrt=lam_inv_sqrt)
    obj = {"sample_x": task.sample_x, "f": task.f, "grad": task.grad,
           "z_from_y": lambda y,x: task.z_from_y(y), "z_dim": d}
    meta = (f"ICNN(2) prelearn {args.icnn_prelearn_steps} steps to quadratic; "
            f"median||y|| target={target:.2f} (obs={med:.2f} → scale={scale:.2f}); dual whitening set; "
            f"loss={args.loss}, δ={args.huber_delta}")
    return obj, meta

def build_task_softplus_pairs(d:int, args, device):
    probe = softplus_probe_y_stats(d, device, x_scale=args.softplus_x_scale,
                                   block=args.softplus_block, nsamp=args.softplus_whiten_samples)
    mu_y = probe["mu"]; cov_y = probe["cov"] + args.softplus_cov_reg * torch.eye(d, device=device)
    lam, U = torch.linalg.eigh(cov_y); lam_inv_sqrt = (1.0 / torch.sqrt(lam.clamp_min(1e-12))).to(device)
    task = SoftplusPairsTask(d=d, device=device, x_scale=args.softplus_x_scale, block=args.softplus_block,
                             y_mu=mu_y, Wy_U=U, Wy_diag_inv_sqrt=lam_inv_sqrt)
    obj = {"sample_x": task.sample_x, "f": task.f, "grad": task.grad,
           "z_from_y": lambda y,x: task.z_from_y(y), "z_dim": d}
    meta = (f"Coupled Soft-plus: x_scale={args.softplus_x_scale}, block={args.softplus_block}; dual whitening set; "
            f"features: z=Wy(y-μ) (+ z^2 in model); loss={args.loss}, δ={args.huber_delta}")
    return obj, meta

TASK_BUILDERS = {
    "quad_spd": build_task_quad_spd,
    "exp_minus_lin": build_task_exp_minus_lin,
    "rand_icnn2": build_task_rand_icnn2,
    "softplus_pairs": build_task_softplus_pairs,
}

# ========================= Training (macro / stream) =============
def train_task(task_name:str, d:int, build_task, args, device):
    obj, meta = build_task(d, args, device)
    zdim = obj["z_dim"]
    width, depth = resnet_size_for_dim(max(d, zdim))
    model = ResNetDLT(in_dim=zdim, width=width, depth=depth,
                      square_feat=True, use_layernorm=False).to(device)
    n_params = sum(p.numel() for p in model.parameters())

    total_steps = (args.steps if d<=10 else
                   int(args.steps*1.5) if d<=20 else
                   int(args.steps*2.0) if d<=50 else
                   int(args.steps*2.5) if d<=100 else
                   int(args.steps*3.0))

    B_macro = macro_batch_size(d)
    if args.sampler == "macro":
        B = B_macro; lr_used = args.lr
    else:
        B = max(1, int(args.mb_factor * B_macro))
        lr_used = args.lr * (B / float(B_macro))

    if task_name == "softplus_pairs" and args.softplus_max_batch > 0:
        B = min(B, args.softplus_max_batch)

    opt = optim.AdamW(model.parameters(), lr=lr_used, weight_decay=1e-4)
    sched = optim.lr_scheduler.CosineAnnealingLR(opt, T_max=total_steps, eta_min=1e-6)
    tscaler = TargetScaler(momentum=0.98)

    print(f"\n================= {task_name} — d={d} =================")
    print(meta)
    print(f"Model:    ResNet(width={width}, depth={depth}), params={n_params/1e6:.2f}M")
    print(f"Sampler:  {args.sampler.upper()} | batch={B} "
          f"({'B_macro' if args.sampler=='macro' else f'{args.mb_factor:.2f}×B_macro'}) "
          f"| steps={total_steps} | lr={lr_used:.2e}")

    best_rmse = float("inf"); best_state = None
    t0 = time.time()
    for step in range(1, total_steps+1):
        y, t, x = compute_macro_batch(obj, B)    # detached; no grad through f
        z = obj["z_from_y"](y, x)
        tzn, _, _ = tscaler.fit_batch(t)

        pred = model(z)
        loss = huber_loss(pred, tzn, delta=args.huber_delta) if args.loss == "huber" \
               else torch.mean((pred - tzn)**2)

        opt.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0 if d <= 20 else 0.5)
        opt.step(); sched.step()

        if step % max(1, total_steps//20) == 0 or step == total_steps:
            ev = evaluate_generic(model, task_name, obj, lambda yy,xx: obj["z_from_y"](yy,xx),
                                  tscaler, device, zdim, test_B=max(8192, 2*B_macro//3))
            flag = ""
            if ev["rmse"] < best_rmse:
                best_rmse = ev["rmse"]; best_state = {k:v.detach().clone() for k,v in model.state_dict().items()}
                flag = " *best"
            print(f"[{100.0*step/total_steps:5.1f}%] train loss={loss.item():.3e} | "
                  f"eval RMSE={ev['rmse']:.3e} (rel={ev['rel_rmse']:.3e}, tstd={ev['t_std']:.3e}){flag}")

    if best_state is not None:
        model.load_state_dict(best_state)

    total_time = time.time() - t0
    final = evaluate_generic(model, task_name, obj, lambda yy,xx: obj["z_from_y"](yy,xx),
                             tscaler, device, zdim, test_B=max(16384, B_macro))
    print(f"Final: RMSE={final['rmse']:.3e} (rel={final['rel_rmse']:.3e}) | time={total_time:.1f}s")
    return {"task":task_name, "d":d, "rmse":final["rmse"], "rel_rmse":final["rel_rmse"],
            "time":total_time, "params":n_params}

# ========================= Aggregation utils =====================
def _agg_mean_std(xs:List[float])->Tuple[float,float]:
    if not xs: return float("nan"), float("nan")
    if len(xs)==1: return xs[0], 0.0
    arr = np.array(xs, dtype=np.float64)
    return float(arr.mean()), float(arr.std(ddof=1))

def print_aggregate_table(rows:List[Dict]):
    # rows: dict(task,d,rmse,rel_rmse,time,params,trial)
    key_to_vals = {}
    for r in rows:
        key = (r["task"], r["d"])
        key_to_vals.setdefault(key, {"rmse":[], "rel":[], "time":[], "params":r["params"]})
        key_to_vals[key]["rmse"].append(r["rmse"])
        key_to_vals[key]["rel"].append(r["rel_rmse"])
        key_to_vals[key]["time"].append(r["time"])
    print("\n========================= AGGREGATED (mean ± sd over trials) =========================")
    print(f"{'task':>14} | {'d':>4} | {'RMSE mean±sd':>22} | {'relRMSE mean±sd':>22} | {'time mean±sd (s)':>20}")
    print("-"*92)
    for (task,d), vals in sorted(key_to_vals.items(), key=lambda k:(k[0][0], k[0][1])):
        m_rmse, s_rmse = _agg_mean_std(vals["rmse"])
        m_rel,  s_rel  = _agg_mean_std(vals["rel"])
        m_t,    s_t    = _agg_mean_std(vals["time"])
        print(f"{task:>14} | {d:4d} | {m_rmse:10.3e} ± {s_rmse:10.3e} | {m_rel:10.3e} ± {s_rel:10.3e} | {m_t:7.1f} ± {s_t:7.1f}")

# ========================= Orchestrators =========================
def build_parser():
    ap = argparse.ArgumentParser()
    ap.add_argument("--functions", nargs="+", default=["quad_spd","exp_minus_lin","rand_icnn2","softplus_pairs"])
    ap.add_argument("--dims", nargs="+", type=int, default=[10,20,50])
    ap.add_argument("--trials", type=int, default=5, help="number of independent runs per (function, d)")
    ap.add_argument("--steps", type=int, default=60000)
    ap.add_argument("--lr", type=float, default=1e-3)
    ap.add_argument("--seed", type=int, default=42)
    ap.add_argument("--spd_norm", choices=["spectral","mean"], default="spectral")
    ap.add_argument("--ridge", type=float, default=1e-2)
    # ICNN pre-learn
    ap.add_argument("--icnn_prelearn_steps", type=int, default=500)
    ap.add_argument("--icnn_prelearn_lr", type=float, default=1e-3)
    ap.add_argument("--icnn_prelearn_batch", type=int, default=2048)
    ap.add_argument("--icnn_prelearn_spd_norm", choices=["spectral","mean"], default="spectral")
    ap.add_argument("--icnn_prelearn_ridge", type=float, default=1e-2)
    # ICNN conditioning
    ap.add_argument("--icnn_whiten_samples", type=int, default=32768)
    ap.add_argument("--icnn_target_y_med", type=float, default=3.0)
    ap.add_argument("--icnn_cov_reg", type=float, default=1e-3)
    # Soft-plus knobs
    ap.add_argument("--softplus_x_scale", type=float, default=0.5)
    ap.add_argument("--softplus_block", type=int, default=64)
    ap.add_argument("--softplus_max_batch", type=int, default=0)  # 0 => no cap
    ap.add_argument("--softplus_whiten_samples", type=int, default=32768)
    ap.add_argument("--softplus_cov_reg", type=float, default=1e-3)
    # Learner loss
    ap.add_argument("--loss", choices=["mse","huber"], default="huber")
    ap.add_argument("--huber_delta", type=float, default=1.0)
    # Sampler (no dataset/pool ever)
    ap.add_argument("--sampler", choices=["macro","stream"], default="macro",
                    help="macro: fresh macro-batch per step; stream: fresh mini-batch per step")
    ap.add_argument("--mb_factor", type=float, default=0.25,
                    help="mini-batch size as fraction of B(d); used when --sampler=stream")
    return ap

def run_trials(args):
    device = get_device()
    print(f"Device: {device.type}, torch {torch.__version__}")

    all_rows = []
    base_seed = int(args.seed)
    for t in range(args.trials):
        trial_seed = base_seed + 1000003*t
        print(f"\n============================ TRIAL {t+1}/{args.trials} (seed={trial_seed}) ============================")
        set_seed(trial_seed)
        # per-trial args copy so builders see different seeds
        args_t = copy.deepcopy(args)
        args_t.seed = trial_seed

        for fn in args.functions:
            if fn not in TASK_BUILDERS:
                print(f"Skipping unknown function '{fn}'")
                continue
            for d in args.dims:
                out = train_task(fn, d, TASK_BUILDERS[fn], args_t, device)
                out["trial"] = t
                all_rows.append(out)
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

    # Per-trial table
    print("\n========================= PER-TRIAL RESULTS =========================")
    print(f"{'trial':>5} | {'task':>14} | {'d':>4} | {'RMSE':>10} | {'relRMSE':>10} | {'time(s)':>7}")
    print("-"*72)
    for r in all_rows:
        print(f"{r['trial']:5d} | {r['task']:>14} | {r['d']:4d} | {r['rmse']:10.3e} | {r['rel_rmse']:10.3e} | {r['time']:7.1f}")

    # Aggregated mean ± std
    print_aggregate_table(all_rows)
    return all_rows

def run_cli(argv: Optional[List[str]] = None):
    parser = build_parser()
    args, _ = parser.parse_known_args(argv)
    rows = run_trials(args)
    return rows

def run_notebook_trials(functions=("quad_spd","exp_minus_lin","rand_icnn2","softplus_pairs"),
                        dims=(10,20,50),
                        trials=5,
                        steps=60000, lr=1e-3, seed=42,
                        spd_norm="spectral", ridge=1e-2,
                        icnn_prelearn_steps=500, icnn_prelearn_lr=1e-3, icnn_prelearn_batch=2048,
                        icnn_prelearn_spd_norm="spectral", icnn_prelearn_ridge=1e-2,
                        softplus_x_scale=0.5, softplus_block=64, softplus_max_batch=0,
                        softplus_whiten_samples=32768, softplus_cov_reg=1e-3,
                        loss="huber", huber_delta=1.0,
                        sampler="macro", mb_factor=0.25):
    class Args: pass
    args = Args()
    args.functions=list(functions); args.dims=list(dims); args.trials=int(trials)
    args.steps=int(steps); args.lr=float(lr); args.seed=int(seed)
    args.spd_norm=str(spd_norm); args.ridge=float(ridge)
    args.icnn_prelearn_steps=int(icnn_prelearn_steps); args.icnn_prelearn_lr=float(icnn_prelearn_lr)
    args.icnn_prelearn_batch=int(icnn_prelearn_batch); args.icnn_prelearn_spd_norm=str(icnn_prelearn_spd_norm)
    args.icnn_prelearn_ridge=float(icnn_prelearn_ridge)
    args.icnn_whiten_samples=32768; args.icnn_target_y_med=3.0; args.icnn_cov_reg=1e-3
    args.softplus_x_scale=float(softplus_x_scale); args.softplus_block=int(softplus_block)
    args.softplus_max_batch=int(softplus_max_batch); args.softplus_whiten_samples=int(softplus_whiten_samples)
    args.softplus_cov_reg=float(softplus_cov_reg)
    args.loss=str(loss); args.huber_delta=float(huber_delta)
    args.sampler=str(sampler); args.mb_factor=float(mb_factor)

    rows = run_trials(args)
    # programmatic aggregation
    agg = {}
    for r in rows:
        k = (r["task"], r["d"])
        agg.setdefault(k, []).append(r["rmse"])
    agg_stats = {k: {"rmse_mean": float(np.mean(v)), "rmse_std": float(np.std(v, ddof=1)) if len(v)>1 else 0.0}
                 for k,v in agg.items()}
    return rows, agg_stats

# ============================ QUICK SMOKE TEST MAIN ============================
# To run full experiments later, comment out this block and use normal CLI, e.g.:
if __name__ == "__main__":
    # quick_args = [
    #     "--functions", "quad_spd", "exp_minus_lin", "rand_icnn2", "softplus_pairs",
    #     "--dims", "5", "10",
    #     "--trials", "2",                 # small repeats for smoke test
    #     "--steps", "8000",               # shorter than full run
    #     "--lr", "1e-3",
    #     "--sampler", "macro",            # or "stream"
    #     "--mb_factor", "0.25",
    #     # Make ICNN pretrain and whitening lighter for the smoke test
    #     "--icnn_prelearn_steps", "300",
    #     "--icnn_prelearn_lr", "1e-3",
    #     "--icnn_prelearn_batch", "1024",
    #     "--icnn_whiten_samples", "8192",
    #     # Stability knobs
    #     "--spd_norm", "spectral",
    #     "--ridge", "1e-2",
    #     "--loss", "huber",
    #     "--huber_delta", "1.0",
    #     # Soft-plus guards for small GPUs
    #     "--softplus_max_batch", "2048",
    #     "--softplus_block", "48",
    #     # Base seed (each trial gets a large-stride offset)
    #     "--seed", "123"
    # ]
    # run_cli(quick_args)
    run_cli()
# ==============================================================================



Device: cuda, torch 2.8.0+cu126

============================ TRIAL 1/5 (seed=42) ============================

================= quad_spd — d=10 =================
Q stats: ||Q||₂≈1.010, κ=69.94, trace=3.0 | features: z=Q^(-1/2) y (+ z^2 in model)
Model:    ResNet(width=384, depth=4), params=1.20M
Sampler:  MACRO | batch=640 (B_macro) | steps=60000 | lr=1.00e-03
[  5.0%] train loss=1.864e-04 | eval RMSE=2.007e-02 (rel=2.016e-02, tstd=9.951e-01) *best
[ 10.0%] train loss=3.698e-05 | eval RMSE=8.777e-03 (rel=8.918e-03, tstd=9.842e-01) *best
[ 15.0%] train loss=1.130e-04 | eval RMSE=8.545e-03 (rel=8.804e-03, tstd=9.706e-01) *best
[ 20.0%] train loss=4.545e-05 | eval RMSE=1.386e-02 (rel=1.427e-02, tstd=9.714e-01)
[ 25.0%] train loss=6.986e-05 | eval RMSE=1.356e-02 (rel=1.348e-02, tstd=1.006e+00)
[ 30.0%] train loss=2.578e-05 | eval RMSE=4.236e-03 (rel=4.452e-03, tstd=9.513e-01) *best
[ 35.0%] train loss=4.257e-05 | eval RMSE=6.179e-03 (rel=6.274e-03, tstd=9.848e-01)
[ 40.0%] train loss=5.8